In [6]:
import bokeh.io
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from bokeh.io import show
from bokeh.plotting import Figure
from bokeh.models import Plot, Range1d, MultiLine, Line, Circle, HoverTool, BoxZoomTool, ResetTool, PanTool, WheelZoomTool, ColumnDataSource, Legend, LegendItem
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4, Set3, Set3_12

bokeh.io.output_notebook()

Loading BokehJS ...

In [89]:
df = pd.read_csv("data/fullevents.csv")

players = np.unique(df["DestinationPlayerID"][~pd.isnull(df.DestinationPlayerID)])

wins = [1,6,11,14,15,17,18,25,27,30,31,35,36]
ties = [2,8,12,16,19,20,24,33,34,37]
losses = [3,4,5,7,9,10,13,21,22,23,26,28,29,32,38]

events = ['Free Kick', 'Duel', 'Pass', 'Others on the ball', 'Foul', 'Goalkeeper leaving line', 'Offside', 'Save attempt', 'Shot', 'Substitution', 'Interruption']
event_colors = {events[i] : Set3_12[i] for i in range(len(events))}

In [63]:
g = nx.Graph()
#g.add_nodes_from(players)

for i in players:
    #colors of nodes can be changed here
    if 'Husk' in i:
        c = Spectral4[1]
    else: c = Spectral4[0]
    
    if 'Husk' in i:
        g.add_node(i, color=c)

In [71]:
color_list = df.EventType.map(event_colors)

for index, row in df[df.EventType=='Pass'].iterrows():
    if not pd.isnull(row.DestinationPlayerID) and ('Husk' in row.DestinationPlayerID):
        g.add_edge(row.OriginPlayerID, row.DestinationPlayerID, edge_color=color_list[index])

In [72]:
plot = Figure(plot_width=800, plot_height=600,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Passes Among Huskies Players"
plot.xgrid.visible = False
plot.ygrid.visible = False

node_hover_tool = HoverTool(tooltips=[("index", "@index")])
plot.add_tools(node_hover_tool)
#plot.add_tools(node_hover_tool, PanTool(), WheelZoomTool(), ResetTool())

graph_renderer = from_networkx(g, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="color")

graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=2)
plot.renderers.append(graph_renderer)

#for n, c in event_colors.items():
    #plot.line(0, 0, color=c, legend_label=n)

#legend = Legend(items=[LegendItem(label=name, renderers=[Line(line_color=color, line_alpha=0.8, line_width=2)]) for (name, color) in event_colors.items()])

show(plot)

In [85]:
gsub = nx.Graph()

for i in players:
    #colors of nodes can be changed here
    if 'Husk' in i:
        c = Spectral4[1]
        gsub.add_node(i, color=c)
    else: c = Spectral4[0]

In [98]:
color_list = df.EventType.map(event_colors)

for index, row in df[df.EventType=='Substitution'].iterrows():
    if not pd.isnull(row.DestinationPlayerID) and ('Husk' in row.DestinationPlayerID) and (row.MatchID in ties):
        gsub.add_edge(row.OriginPlayerID, row.DestinationPlayerID, edge_color=color_list[index])

In [99]:
plot2 = Figure(plot_width=800, plot_height=600,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot2.title.text = "Substitutions of Huskies Players"
plot2.xgrid.visible = False
plot2.ygrid.visible = False

node_hover_tool = HoverTool(tooltips=[("index", "@index")])
plot2.add_tools(node_hover_tool)
#plot.add_tools(node_hover_tool, PanTool(), WheelZoomTool(), ResetTool())

graph_renderer = from_networkx(gsub, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="color")

graph_renderer.edge_renderer.glyph = MultiLine(line_color="edge_color", line_alpha=0.8, line_width=2)
plot2.renderers.append(graph_renderer)

#for n, c in event_colors.items():
    #plot.line(0, 0, color=c, legend_label=n)

#legend = Legend(items=[LegendItem(label=name, renderers=[Line(line_color=color, line_alpha=0.8, line_width=2)]) for (name, color) in event_colors.items()])

show(plot2)

In [12]:
gDuel = nx.Graph()

for i in players:
    #colors of nodes can be changed here
    if 'Husk' in i:
        c = Spectral4[1]
    else:
        c = Spectral4[0]
        break
    
    gDuel.add_node(i, color=c)
    
    
for index, row in df.iterrows():
    if (row.EventType == "Duel" and df.EventType[index+1] == "Duel"):
        dueler = df.OriginPlayerID[index + 1]
        if ('Husk' in row.OriginPlayerID) and ('Husk' in dueler):
            gDuel.add_edge(row.OriginPlayerID, dueler, edge_color=Spectral4[2])
            
            
        

In [11]:
f2 = Figure(title="Destination Events", plot_width=800, plot_height=600)
f2.grid.visible = False

idx = ~pd.isnull(df.EventDestination_x)

#heatmap, xedges, yedges = np.histogram2d(df.EventOrigin_x[idx], df.EventOrigin_y[idx], bins=50)
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

r, bins = f2.hexbin(df.EventDestination_x[idx], df.EventDestination_y[idx], size=2)

f2.add_tools(HoverTool(
    tooltips=[("count", "@c"), ("(q,r)", "(@q, @r)")],
    mode="mouse", point_policy="follow_mouse", renderers=[r]
))

show(f2)

In [15]:
np.sum(infights['receiving']) / 38

12.605263157894736

In [16]:
saves = pd.DataFrame(index=[p for p in players if 'Husk' in p])
saves['count'] = 0

for (index, row) in df[(df.EventType == 'Duel') & (df.OriginPlayerID.str.contains('Huskies'))].iterrows():
    if df.EventType[index + 1] == 'Duel' and row.OriginPlayerID == df.OriginPlayerID[index + 1] and df.EventType[index + 1] == 'Duel':
        saves['count'][row.OriginPlayerID] += 1

In [20]:
import matplotlib.pyplot as plt
%matplotlib widget

In [21]:
pos = nx.spring_layout(gDuel)
nx.draw(gDuel, pos, with_labels=True, connectionstyle='arc3, rad = 0.1')
#edge_labels=dict([((u,v,),d['length'])
#             for u,v,d in G.edges(data=True)])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …